In [1]:
!pip install openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.9/193.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.4/144.4 kB 6.2 MB/s eta 0:00:00


In [2]:
!pip install openai pydantic typing sendgrid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 51.4 MB/s eta 0:00:00
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26304 sha256=014d26ecec973945c5546561699728566f3e190d0693099cc6ff74d24e03eb64
  Stored in directory: /root/.cache/pip/wheels/12/98/52/2bffe242a9a487f00886e43b8ed8dac46456702e11a0d6abef
Successfully built typing
  Attempting uninstall: cryptography
    Found existing installation: cryptography 43.0.3
    Uninstalling cryptography-43.0.3:
      Successfully uninstalled cryptography-43.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 46.0.1 which is

In [5]:
from agents import Agent, Runner, trace, WebSearchTool, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel
from sendgrid.helpers.mail import Email, To, Content, Mail
from IPython.display import display,Markdown
import sendgrid
import asyncio
import os
from google.colab import userdata
from typing import Dict
from openai import OpenAI

In [2]:
os.environ["OPENAI_API_KEY"]=userdata.get("OPENAI_API_KEY")

In [3]:
search_agent_inst="You are a research assistant agent.Given a search term, you search the web for that term and \
produces a concise summary of results.summary must be of 2-3 paragraphs and less than 300 words. \
Capture the main points. Write Succintly, no need to have complete sentences or good grammer. \
This will be used by someone synthesizing a report, so it's vital you capture the essence and ignore any fluff. \
Do not include any additional commentary other than the summary itself."

search_agent=Agent(
    name="search agent",
    instructions=search_agent_inst,
    tools=[WebSearchTool(search_context_size='low')],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required")
)

In [11]:
message="Latest AI Agents frameworks in 2025"
with trace("Web Search"):
  result=await Runner.run(search_agent, message)

display(Markdown(result.final_output))

In 2025, several AI agent frameworks have emerged, each offering unique capabilities:

- **Agent Lightning**: A flexible framework enabling reinforcement learning-based training of large language models (LLMs) for any AI agent. It decouples agent execution from training, allowing seamless integration with existing agents developed through various methods. ([arxiv.org](https://arxiv.org/abs/2508.03680?utm_source=openai))

- **Polymorphic Combinatorial Framework (PCF)**: Utilizes LLMs and mathematical frameworks to guide the design of adaptive AI agents for complex, dynamic environments. PCF enables real-time parameter reconfiguration through combinatorial spaces, allowing agents to adapt their core behavioral traits dynamically. ([arxiv.org](https://arxiv.org/abs/2508.01581?utm_source=openai))

- **AutoAgent**: A fully-automated, zero-code framework that enables users to create and deploy LLM agents through natural language alone. It operates as an autonomous agent operating system, comprising components like Agentic System Utilities and an LLM-powered Actionable Engine. ([arxiv.org](https://arxiv.org/abs/2502.05957?utm_source=openai))

- **LightAgent**: A lightweight yet powerful agentic framework that integrates core functionalities such as Memory, Tools, and Tree of Thought, while maintaining an extremely lightweight structure. It seamlessly integrates with mainstream chat platforms, enabling developers to build self-learning agents. ([arxiv.org](https://arxiv.org/abs/2509.09292?utm_source=openai))

- **Amazon Bedrock AgentCore**: A platform designed to simplify the development and deployment of advanced AI agents. It includes modular services supporting the full production lifecycle, such as AgentCore Runtime for scalable serverless deployment and AgentCore Memory for context management. ([techradar.com](https://www.techradar.com/pro/aws-looks-to-super-charge-ai-agents-with-amazon-bedrock-agentcore?utm_source=openai))

- **Model Context Protocol (MCP)**: An open standard introduced by Anthropic to standardize the integration and sharing of data between AI systems like LLMs and external tools. Adopted by major AI providers, including OpenAI and Google DeepMind, MCP provides a universal interface for reading files, executing functions, and handling contextual prompts. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Model_Context_Protocol?utm_source=openai))

- **Manus**: China's fully autonomous AI agent capable of operating independently based on LLMs. It can autonomously handle complex tasks, including writing and deploying code, marking a significant advancement in AI agent capabilities. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Manus_%28AI_agent%29?utm_source=openai))

These frameworks represent the forefront of AI agent development in 2025, each contributing to the evolution of intelligent, autonomous systems. 

# To get different terms for web search for a query to get best answer we use ***Structured Output*** with pydantic

In [4]:
no_of_search_terms=3

planner_inst=f"You are a helpful research assistant agent. Given a query, come up with a set of web searches \
to perform to get best answer of query. output {no_of_search_terms} terms to query for."

class websearchitem(BaseModel):
  reason:str
  "Your reasoning for why this search is important to query."

  query:str
  "the search term to use for web search."

class websearchplan(BaseModel):
  searches:list[websearchitem]
  "A list of web searches to perform to best answer the query"

planner_agent=Agent(
    name="Planner Agent",
    instructions=planner_inst,
    model="gpt-4o-mini",
    output_type=websearchplan
)

In [18]:
message="Latest AI Agents frameworks in 2025"
with trace("planner Search terms"):
  result=await Runner.run(planner_agent, message)
  print(result.final_output)

searches=[websearchitem(reason='To find updates on the newest AI agent frameworks being developed or released in 2025.', query='latest AI agent frameworks 2025'), websearchitem(reason='To examine industry reports and analyses on AI agent frameworks that emerged in 2025.', query='AI agent frameworks trends 2025'), websearchitem(reason='To identify specific tools or platforms that focus on AI agents released in 2025.', query='new AI agent tools 2025')]


In [6]:
@function_tool
def send_email(subject:str, html_body:str)-> Dict[str,str]:
  """Send out an email with given subject and HTML body"""
  sg=sendgrid.SendGridAPIClient(api_key=userdata.get("SENDGRID_API_KEY"))

  from_email=Email("muasif025@gmail.com")
  to_email=To("as29624041@gmail.com")
  content=Content("text/html",html_body)
  mail=Mail(from_email,to_email,subject,content).get()

  response=sg.client.mail.send.post(request_body=mail)

  return {"status":"success"}


In [9]:
email_agent_inst="""You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided  with a detailed report. You should use your send_email tool to send one email,
providing the report converted into clean, well presented HTML with an appropriate subject line."""

email_agent=Agent(
    name="Email Agent",
    instructions=email_agent_inst,
    tools=[send_email],
    model="gpt-4o-mini"
)

In [12]:
writer_agent_inst="You are a senior researcher agent tasked with writing a cohesive report for research query. \
You will be given with original query and some intial research done by a research asstant. \
You should first come up with an outline for the report that describes the structure and \
flow of the report.Then, generate the report and return that as your final report. \
The final report should be in markdoen format, and it should be lengthy and detailed. \
Aim for 5-10 pages of content, at least 1000 words."

class ReportData(BaseModel):
  short_summary:str
  """A short 2-3 sentences summary of the findings."""
  markdown_report:str
  """The final Markdown format Report"""
  follow_up_questions:list[str]
  """Suggested Topics to research further."""

writer_agent=Agent(
    name="WriterAgent",
    instructions=writer_agent_inst,
    model="gpt-4o-mini",
    output_type=ReportData
)

In [16]:
async def plan_searches(query: str):
  """Use the planner agent to plan which searches to run for query"""
  print("Planning Searhes......")
  result=await Runner.run(planner_agent, query)
  return result.final_output

async def perform_searches(search_plan: websearchplan):
    print("Searching...........")
    tasks=[asyncio.create_task(search(item)) for item in search_plan.searches] # Iterate over search_plan.searches
    results=await asyncio.gather(*tasks)
    print("Finished Searching.....")
    return results

async def search(item: websearchitem):
  input=f"Search term: {item.query}\n Reason for searching: {item.reason}"
  result=await Runner.run(search_agent, input)
  return result.final_output

async def write_report(query: str, search_results: list[str]):
  print("Thinking about Report......")
  input=f"Original query: {query}\nSummarized search results: {search_results}"
  result=await Runner.run(writer_agent, input)
  print("Finished Writing Report.....")
  return result.final_output

async def email_sender(report: ReportData):
  print("Writing Email........")
  result=await Runner.run(email_agent, report.markdown_report)
  print("Email sent....")
  return report

In [17]:
message="Latest AI Agents frameworks in 2025"

with trace("Planning Searches"):
  print("Searching start......")
  search_plan=await plan_searches(message)
  search_results=await perform_searches(search_plan)
  report=await write_report(message, search_results)
  await email_sender(report)
  print("All done!!!!!!")

Searching start......
Planning Searhes......
Searching...........
Finished Searching.....
Thinking about Report......
Finished Writing Report.....
Writing Email........
Email sent....
All done!!!!!!
